In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import time

import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

<IPython.core.display.Javascript object>

In [3]:
torch.manual_seed(0)

<IPython.core.display.Javascript object>

## Set up the neural network

Create a class for network structure.

Create helper functions to initialize the weights and to evaluate the training and testing functions.

In [4]:
class My_Net(nn.Module):
    def __init__(self, n_features, n_classes, hidden_dim1=15, hidden_dim2=15):

        super(My_Net, self).__init__()

        self.layer_1 = nn.Linear(n_features, hidden_dim1)
        self.layer_2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.layer_3 = nn.Linear(hidden_dim2, n_classes)

        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_data):
        out = self.layer_1(input_data)
        out = self.relu(out)
        out = self.layer_2(out)
        out = self.relu(out)
        out = self.layer_3(out)
        out = self.softmax(out)
        return out

<IPython.core.display.Javascript object>

In [5]:
def weights_init(m):
    """Initialize the network using Xavier initialization."""
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight.data)

<IPython.core.display.Javascript object>

In [6]:
def train_eval(net, train_loader, loss, verbose=True):

    correct = 0
    total = 0
    loss_sum = 0
    num_batches = 0
    for inputs, labels in train_loader:
        outputs = net(inputs)
        predicted = torch.argmax(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted.int() == labels.int()).sum()
        loss_sum += loss(outputs, labels).item()
        num_batches += 1

    if verbose:
        print(f"Train accuracy: {100 * correct.item() / total:.3f}%")

    return loss_sum / num_batches, correct.item() / total

<IPython.core.display.Javascript object>

In [7]:
def test_eval(net, test_loader, loss, verbose=True):

    correct = 0
    total = 0
    loss_sum = 0
    num_batches = 0
    for inputs, labels in test_loader:
        outputs = net(inputs)
        predicted = torch.argmax(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted.int() == labels.int()).sum()
        loss_sum += loss(outputs, labels).item()
        num_batches += 1

    if verbose:
        print(f"Test accuracy: {100 * correct.item() / total:.3f}%")

    return loss_sum / num_batches, correct.item() / total

<IPython.core.display.Javascript object>

In [8]:
def main(train_loader, test_loader, n_features, n_classes, epochs=25, lr=1e-2):

    # Initialize the neural network
    net = My_Net(n_features, n_classes)
    loss = nn.CrossEntropyLoss()
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    net.apply(weights_init)

    train_loss_store = []
    train_acc_store = []
    test_loss_store = []
    test_acc_store = []

    for epoch in range(epochs):
        time1 = time.time()
        print(f"Epoch {epoch + 1}:")
        for i, (feat, label) in enumerate(train_loader, 0):
            opt.zero_grad()
            outputs = net(feat)
            l = loss(outputs, label)
            l.backward()
            opt.step()

        l_temp, acc_temp = train_eval(net, train_loader, loss)
        train_loss_store.append(l_temp)
        train_acc_store.append(acc_temp)

        l_temp, acc_temp = test_eval(net, test_loader, loss)
        test_loss_store.append(l_temp)
        test_acc_store.append(acc_temp)

        time2 = time.time()
        print(f"Time lapse: {time2-time1:.2f} sec \n")

<IPython.core.display.Javascript object>

## Make a helper function to parse the data file

In [9]:
def parse_data(fpath):

    # Input the data and split into features and labels
    data = np.genfromtxt(fpath, dtype=np.int64, skip_header=7)
    X, y = data[:, :-1], data[:, -1]
    n_classes = len(np.unique(y))
    n_features = X.shape[1]

    # Index classes starting at 0 for CrossEntropyLoss() to work
    y -= 1

    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    # Convert to tensors
    X_train = torch.Tensor(X_train)
    X_test = torch.Tensor(X_test)
    y_train = torch.LongTensor(y_train)
    y_test = torch.LongTensor(y_test)

    # Pick optimal batch size
    if "stable4" in fpath:
        batch_size = 1
    elif "stable5" in fpath:
        batch_size = 32
    elif "stable6" in fpath:
        batch_size = 64

    # Make data loaders for the test and training set
    train = torch.utils.data.TensorDataset(X_train, y_train)
    test = torch.utils.data.TensorDataset(X_test, y_test)

    train_loader = torch.utils.data.DataLoader(
        train, batch_size=batch_size, num_workers=2
    )
    test_loader = torch.utils.data.DataLoader(
        test, batch_size=batch_size, shuffle=False, num_workers=2
    )

    return train_loader, test_loader, n_features, n_classes

<IPython.core.display.Javascript object>

## Train the model for each dataset

In [10]:
fpath = "stable4.txt"
train_loader, test_loader, n_features, n_classes = parse_data(fpath)
main(train_loader, test_loader, n_features, n_classes)

Epoch 1:
Train accuracy: 63.636%
Test accuracy: 60.000%
Time lapse: 0.23 sec 

Epoch 2:
Train accuracy: 63.636%
Test accuracy: 60.000%
Time lapse: 0.16 sec 

Epoch 3:
Train accuracy: 75.000%
Test accuracy: 60.000%
Time lapse: 0.18 sec 

Epoch 4:
Train accuracy: 79.545%
Test accuracy: 60.000%
Time lapse: 0.16 sec 

Epoch 5:
Train accuracy: 79.545%
Test accuracy: 60.000%
Time lapse: 0.16 sec 

Epoch 6:
Train accuracy: 81.818%
Test accuracy: 60.000%
Time lapse: 0.17 sec 

Epoch 7:
Train accuracy: 88.636%
Test accuracy: 70.000%
Time lapse: 0.17 sec 

Epoch 8:
Train accuracy: 93.182%
Test accuracy: 70.000%
Time lapse: 0.17 sec 

Epoch 9:
Train accuracy: 93.182%
Test accuracy: 75.000%
Time lapse: 0.16 sec 

Epoch 10:
Train accuracy: 95.455%
Test accuracy: 80.000%
Time lapse: 0.16 sec 

Epoch 11:
Train accuracy: 95.455%
Test accuracy: 80.000%
Time lapse: 0.15 sec 

Epoch 12:
Train accuracy: 95.455%
Test accuracy: 80.000%
Time lapse: 0.15 sec 

Epoch 13:
Train accuracy: 95.455%
Test accuracy: 

<IPython.core.display.Javascript object>

In [11]:
fpath = "stable5.txt"
train_loader, test_loader, n_features, n_classes = parse_data(fpath)
main(train_loader, test_loader, n_features, n_classes)

Epoch 1:
Train accuracy: 55.028%
Test accuracy: 57.143%
Time lapse: 0.15 sec 

Epoch 2:
Train accuracy: 68.017%
Test accuracy: 63.312%
Time lapse: 0.14 sec 

Epoch 3:
Train accuracy: 76.955%
Test accuracy: 68.831%
Time lapse: 0.15 sec 

Epoch 4:
Train accuracy: 78.911%
Test accuracy: 72.403%
Time lapse: 0.14 sec 

Epoch 5:
Train accuracy: 81.006%
Test accuracy: 72.078%
Time lapse: 0.14 sec 

Epoch 6:
Train accuracy: 82.263%
Test accuracy: 74.026%
Time lapse: 0.13 sec 

Epoch 7:
Train accuracy: 81.844%
Test accuracy: 73.701%
Time lapse: 0.14 sec 

Epoch 8:
Train accuracy: 82.402%
Test accuracy: 74.351%
Time lapse: 0.13 sec 

Epoch 9:
Train accuracy: 83.380%
Test accuracy: 75.649%
Time lapse: 0.14 sec 

Epoch 10:
Train accuracy: 83.240%
Test accuracy: 76.623%
Time lapse: 0.14 sec 

Epoch 11:
Train accuracy: 83.520%
Test accuracy: 75.325%
Time lapse: 0.14 sec 

Epoch 12:
Train accuracy: 83.659%
Test accuracy: 76.299%
Time lapse: 0.15 sec 

Epoch 13:
Train accuracy: 84.916%
Test accuracy: 

<IPython.core.display.Javascript object>

In [12]:
fpath = "stable6.txt"
train_loader, test_loader, n_features, n_classes = parse_data(fpath)
main(train_loader, test_loader, n_features, n_classes)

Epoch 1:
Train accuracy: 66.844%
Test accuracy: 66.168%
Time lapse: 0.72 sec 

Epoch 2:
Train accuracy: 66.844%
Test accuracy: 66.168%
Time lapse: 0.78 sec 

Epoch 3:
Train accuracy: 66.844%
Test accuracy: 66.168%
Time lapse: 0.78 sec 

Epoch 4:
Train accuracy: 70.781%
Test accuracy: 70.766%
Time lapse: 0.75 sec 

Epoch 5:
Train accuracy: 71.683%
Test accuracy: 71.580%
Time lapse: 0.72 sec 

Epoch 6:
Train accuracy: 72.769%
Test accuracy: 72.424%
Time lapse: 0.72 sec 

Epoch 7:
Train accuracy: 73.706%
Test accuracy: 73.146%
Time lapse: 0.73 sec 

Epoch 8:
Train accuracy: 73.863%
Test accuracy: 73.512%
Time lapse: 0.84 sec 

Epoch 9:
Train accuracy: 73.955%
Test accuracy: 73.645%
Time lapse: 0.72 sec 

Epoch 10:
Train accuracy: 73.567%
Test accuracy: 73.136%
Time lapse: 0.72 sec 

Epoch 11:
Train accuracy: 74.631%
Test accuracy: 73.848%
Time lapse: 0.71 sec 

Epoch 12:
Train accuracy: 74.548%
Test accuracy: 73.736%
Time lapse: 0.70 sec 

Epoch 13:
Train accuracy: 75.372%
Test accuracy: 

<IPython.core.display.Javascript object>